In [ ]:
from bertopic import BERTopic
from collections import Counter
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
en_BW = pd.read_csv('../data/BW_tweets_all_Topic.csv')
docs = list(en_BW.full_text)

hydratedFile = pd.read_json('../data/idtweetVALUES.jsonl',lines=True)
notes = pd.read_csv('../data/notes-00000.tsv',delimiter='\t')

In [ ]:
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topic_model=topic_model.load('../model/BW_BertTopic.model')

In [ ]:
def GetTopicDict(file='../data/Manual_Topic_Label_Done.csv'):
    df = pd.read_csv(file)
    d = dict(zip(df['Topic ID'],df['Generic Label']))
    return d
TopicId2Label=GetTopicDict()

In [ ]:
hts=list(set(hydratedFile.id))
BW_tweets = list(set(notes.tweetId))

In [ ]:
hydratedFile['Month Name']= hydratedFile['created_at'].apply(lambda x: x.month_name())
hydratedFile['Month']= hydratedFile['created_at'].apply(lambda x: x.month)
hydratedFile['Year']= hydratedFile['created_at'].apply(lambda x: x.year)
hydratedFile['created_at'].apply(lambda x: x.year).value_counts()

In [ ]:
docs = list(hydratedFile.full_text)
docs = [str(x) for x in docs]

In [ ]:
predicted_topics,_ = topic_model.transform(docs)
predicted_topic_names=[TopicId2Label[x] for x in  predicted_topics]

In [ ]:
en_BW['Topic']=predicted_topic_names

In [ ]:
en_BW.to_csv('CR_Updated_Topic_Predicted.csv',index=False)

In [ ]:
hydratedFile['Topic']=predicted_topic_names
hf_df = hydratedFile[['Topic','Month','Year']]
hf_df=hf_df[hf_df.Year==2021]
hf_df=hf_df[hf_df.Topic !='Outlier']

In [ ]:

sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

# Initialize the FacetGrid object
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
g = sns.FacetGrid(hf_df, row="Topic", hue="Topic", aspect=15, height=.85, palette=pal)

# Draw the densities in a few steps
g.map(sns.kdeplot, "Month",
       clip_on=False,bw_adjust=.5,
      fill=True, alpha=1, linewidth=1.5)
g.map(sns.kdeplot, "Month", clip_on=False,bw_adjust=.5, color="w", lw=2)

# passing color=None to refline() uses the hue mapping
g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)


# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)

g.map(label, "Month")

# Set the subplots to overlap
g.figure.subplots_adjust(hspace=-.25)
# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(yticks=[], ylabel="")
g.despine(bottom=True, left=True)

# Claim Review Topic Distribution


In [ ]:
claim_review_file = "../data/UpdatedClaimReview/2022_01_14/claim_reviews.json"
facts = pd.read_json(claim_review_file)

In [ ]:
facts['CR_time']=facts.reviews.apply(lambda x: list(set([xx['date_published'] for xx in x]))[0])
facts['text']=facts.claim_text.apply(lambda x:" ".join(x))
facts = facts.drop_duplicates(subset=['text'])
facts=facts.dropna()

In [ ]:
len(facts)

In [ ]:
facts['Month']=facts.CR_time.apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d').month  if x else np.nan)
facts['Year']=facts.CR_time.apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d').year if x else np.nan )

In [ ]:
facts['Year'].value_counts()

In [ ]:
docs = list(facts['text'])
predicted_topics,_ = topic_model.transform(docs)
predicted_topic_names=[TopicId2Label[x] for x in  predicted_topics]
facts['Topic']=predicted_topic_names

In [ ]:

sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})


for year in [2021]:#[2017,2018,2019,2020,2021]:
    d=facts[facts.Year==year]
    hf_df = d[['Topic','Month']]
    hf_df=hf_df[hf_df.Topic!='Outlier']
    hf_df_2 = hf_df[hf_df.apply(lambda x: True if x.Topic in['Health','Politics','Social Media','Economy','Society','Country','Sports'] else False,axis=1)]

    # Initialize the FacetGrid object
    pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
    g = sns.FacetGrid(hf_df_2, row="Topic", hue="Topic", aspect=15, height=.85, palette=pal)

    # Draw the densities in a few steps
    g.map(sns.kdeplot, "Month",
        clip_on=False,bw_adjust=.5,
        fill=True, alpha=1, linewidth=1.5)
    g.map(sns.kdeplot, "Month", clip_on=False,bw_adjust=.5, color="w", lw=2)

    # passing color=None to refline() uses the hue mapping
    g.refline(y=0, linewidth=2, linestyle="-", color=None, clip_on=False)


    # Define and use a simple function to label the plot in axes coordinates
    def label(x, color, label):
        ax = plt.gca()
        ax.text(0, .2, label, fontweight="bold", color=color,
              ha="left", va="center", transform=ax.transAxes)


    g.map(label, "Month")
    g.fig.suptitle('Claim Review 2021')

    # Set the subplots to overlap
    g.figure.subplots_adjust(hspace=-.25)
    # Remove axes details that don't play well with overlap
    g.set_titles("")
    g.set(yticks=[], ylabel="")
    for ax in g.axes.flat:
        ax.set_xticks(range(15))
        ax.set_xticklabels(['','Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec','',''])
        #ax.set_xticklabels(['','Jan','Feb','Mar','Apr','','','','','','','','','',''])
    g.despine(bottom=True, left=True)

# Training + Visualisation

In [ ]:
en_BW=pd.read_csv('../data/BW_tweets_all_Topic.csv')
docs = list(en_BW.full_text)

In [ ]:
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

In [ ]:
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.save("../model/BW_BertTopic.model")

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.get_topic(2)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(10)

In [ ]:
topic_model.get_topic(0) 

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_distribution(probs[0], min_probability=0.015)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)

In [ ]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)